In [1]:
import cv2
import mediapipe as mp
import numpy as np
import pandas as pd
import os
import time

In [2]:
mp_hands=mp.solutions.hands
hands=mp_hands.Hands(static_image_mode=False,max_num_hands=1)
mp_drawing=mp.solutions.drawing_utils

In [3]:

# Data list
data = []
num_signs = 5
frames_per_sign = 25

video = cv2.VideoCapture(0)
if not video.isOpened():
    print("❌ Error: Could not open webcam.")
    exit()

for i in range(num_signs):
    label = input(f"Enter label for sign {i+1}/{num_signs}: ")
    print(f"📸 Collecting {frames_per_sign} frames for '{label}'. Please show the sign...")

    frame_count = 0
    while frame_count < frames_per_sign:
        ret, frame = video.read()
        if not ret:
            print("❌ Failed to grab frame.")
            break

        image = cv2.flip(frame, 1)
        img_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        result = hands.process(img_rgb)

        if result.multi_hand_landmarks:
            for hand_landmarks in result.multi_hand_landmarks:
                mp_drawing.draw_landmarks(image, hand_landmarks, mp_hands.HAND_CONNECTIONS)

                landmarks = []
                for lm in hand_landmarks.landmark:
                    landmarks.extend([lm.x, lm.y, lm.z])

                data.append([label] + landmarks)
                frame_count += 1
                print(f"✔️ Captured frame {frame_count}/{frames_per_sign} for '{label}'")
                time.sleep(1)  # Slight delay between captures

        cv2.imshow("Sign Capture", image)
        if cv2.waitKey(1) & 0xFF == ord("q"):
            print("❎ Quit early.")
            video.release()
            cv2.destroyAllWindows()
            exit()

print("✅ All sign data collected.")

video.release()
cv2.destroyAllWindows()

# Save to CSV
if data:
    df = pd.DataFrame(data)
    df.to_csv("sign_data.csv", mode='a', header=not os.path.exists("sign_data.csv"), index=False)
    print("✅ All data saved to sign_data.csv.")
else:
    print("⚠️ No data was saved.")


📸 Collecting 25 frames for 'hello'. Please show the sign...
✔️ Captured frame 1/25 for 'hello'
✔️ Captured frame 2/25 for 'hello'
✔️ Captured frame 3/25 for 'hello'
✔️ Captured frame 4/25 for 'hello'
✔️ Captured frame 5/25 for 'hello'
✔️ Captured frame 6/25 for 'hello'
✔️ Captured frame 7/25 for 'hello'
✔️ Captured frame 8/25 for 'hello'
✔️ Captured frame 9/25 for 'hello'
✔️ Captured frame 10/25 for 'hello'
✔️ Captured frame 11/25 for 'hello'
✔️ Captured frame 12/25 for 'hello'
✔️ Captured frame 13/25 for 'hello'
✔️ Captured frame 14/25 for 'hello'
✔️ Captured frame 15/25 for 'hello'
✔️ Captured frame 16/25 for 'hello'
✔️ Captured frame 17/25 for 'hello'
✔️ Captured frame 18/25 for 'hello'
✔️ Captured frame 19/25 for 'hello'
✔️ Captured frame 20/25 for 'hello'
✔️ Captured frame 21/25 for 'hello'
✔️ Captured frame 22/25 for 'hello'
✔️ Captured frame 23/25 for 'hello'
✔️ Captured frame 24/25 for 'hello'
✔️ Captured frame 25/25 for 'hello'
📸 Collecting 25 frames for 'victory'. Please show

In [4]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import joblib

# Load data
df = pd.read_csv("sign_data.csv")

# Remove any rows where all values are not numeric or contain NaNs
df = df.dropna()  # Drop rows with NaNs

# OPTIONAL: Remove header rows mistakenly appended as data
df = df[df.iloc[:, 0] != 'label']  # If 'label' is repeated due to appending

# Reset index after cleaning
df.reset_index(drop=True, inplace=True)

# Convert all data to proper types (in case label column is object)
X = df.iloc[:, 1:].astype(float).values  # Convert features to float
y = df.iloc[:, 0].values                 # Labels (keep as is if string/class)

# Train/test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1)

# Model training
model = RandomForestClassifier()
model.fit(X_train, y_train)

# Prediction and accuracy
y_pred = model.predict(X_test)
print("✅ Model accuracy:", accuracy_score(y_test, y_pred))

# Save the model
joblib.dump(model, "sign_model.pkl")
print("✅ Model saved to sign_model.pkl")


✅ Model accuracy: 0.9574468085106383
✅ Model saved to sign_model.pkl


In [5]:

# Load trained model
model = joblib.load("sign_model.pkl")

# Initialize MediaPipe


# Start webcam
video = cv2.VideoCapture(0)

while video.isOpened():
    ret, frame = video.read()
    if not ret:
        break

    image = cv2.flip(frame, 1)
    rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    results = hands.process(rgb)

    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            mp_drawing.draw_landmarks(image, hand_landmarks, mp_hands.HAND_CONNECTIONS)

            landmarks = []
            for lm in hand_landmarks.landmark:
                landmarks.extend([lm.x, lm.y, lm.z])

            if len(landmarks) == 63:
                prediction = model.predict([landmarks])[0]
                cv2.putText(image, prediction, (10, 70), cv2.FONT_HERSHEY_SIMPLEX,
                            2, (255, 0, 0), 3, cv2.LINE_AA)

    cv2.imshow("Real-Time Sign Detection", image)
    if cv2.waitKey(10) & 0xFF == ord('q'):
        break

video.release()
cv2.destroyAllWindows()
